In [1]:
import numpy as np
import pandas as pd
import qPAI_cINN_uncertainty_estimation.config as c
import matplotlib.pyplot as plt

CPU only


In [4]:
output_csv = c.output_dir / 'multiple_eval.csv'
df = pd.read_csv(output_csv)

In [5]:
print(df)

     Unnamed: 0                         name partitioned_eval  sparsity_eval  \
0             0        phantom_3_partitioned             part              3   
1             1        phantom_3_partitioned             part              5   
2             2        phantom_3_partitioned             part             10   
3             3        phantom_3_partitioned             part             25   
4             4        phantom_3_partitioned             part             40   
..          ...                          ...              ...            ...   
235         235  generic_flexi_unpartitioned           unpart              3   
236         236  generic_flexi_unpartitioned           unpart              5   
237         237  generic_flexi_unpartitioned           unpart             10   
238         238  generic_flexi_unpartitioned           unpart             25   
239         239  generic_flexi_unpartitioned           unpart             40   

     med_calib_err  calib_err_68  med_u

In [23]:
df_phantom = df[df.name.str.startswith('p')]
df_generic = df[df.name.str.startswith('g')]
df_melanin = df[df.name.str.startswith('m')]

In [92]:
df = pd.read_csv(output_csv)
df = df[df.name.str.startswith('p')]  # Phantom only
#df = df[~df.name.str.contains('flexi')]  # Non-flexi only
phantom_best_rows_part = []
for train in ['_partitioned', 'unpartitioned']:
    for eval_part in [['part'], ['unpart']]:
        for sparsity in [3, 5, 10, 25, 40]:
            row = df[(df.name.str.contains(train)) & (df.partitioned_eval.isin(eval_part)) & (df.sparsity_eval == sparsity)].sort_values(by='med_rel_err').head(1).to_dict()
            row = {key: [sub_val for sub_val in val.values()][0] for key, val in row.items()}
            phantom_best_rows_part.append(row)
df = pd.DataFrame(phantom_best_rows_part)
df['Training'] = df.name.str.split('_').str[2]
df['Evaluation'] = df.partitioned_eval
df['Sparsity'] = df.sparsity_eval
df = df.round(1)
df['Median relative error and IQR'] = df.med_rel_err.astype(str) + '% (' + df.iqr_lower.astype(str) + ', ' + df.iqr_upper.astype(str) + ')%'
df.pivot(index='Sparsity', columns=['Training', 'Evaluation'], values=['Median relative error and IQR'])

Median relative error and IQR                      \
Training                     partitioned                       
Evaluation                          part              unpart   
Sparsity                                                       
3                     15.2% (6.7, 37.6)%  22.5% (9.8, 52.8)%   
5                     11.2% (4.5, 30.9)%  16.1% (6.5, 38.9)%   
10                     3.6% (1.4, 15.3)%   8.9% (3.5, 26.3)%   
25                      1.6% (0.5, 6.2)%   3.0% (1.1, 13.0)%   
40                      1.6% (0.5, 6.7)%    1.6% (0.5, 7.6)%   

                                                    
Training         unpartitioned                      
Evaluation                part              unpart  
Sparsity                                            
3           18.1% (8.5, 40.7)%  19.3% (8.6, 44.2)%  
5           10.2% (3.9, 34.3)%  11.6% (4.1, 34.5)%  
10           4.4% (1.7, 17.0)%   6.6% (2.2, 21.9)%  
25            2.0% (0.7, 9.9)%    2.4% (0.8, 7.9)%  
40            1.4% (0.5, 5.7)%    1.3% (0.4, 5.2)%

In [90]:
df = pd.read_csv(output_csv)